先理解下概念，structured_outputs 具体指什么？

**Have a model return output that matches a specific schema.**

让大模型的响应符合规定的范式。

几乎所有的大模型都支持这个功能，比如 openai：[Structured Outputs](https://platform.openai.com/docs/guides/structured-outputs)

首推 `.with_structured_output()` 方法。

`with_structured_output()` is implemented for models that provide native APIs for structuring outputs, like tool/function calling or JSON mode, and makes use of these capabilities under the hood.

推荐使用 Pydantic 来书写范式。

If we want the model to return a Pydantic object, we just need to pass in the desired Pydantic class. The key advantage of using Pydantic is that the model-generated output will be validated. Pydantic will raise an error if any required fields are missing or if any fields are of the wrong type.

**提示**：Beyond just the structure of the Pydantic class, the name of the Pydantic class, the docstring, and the names and provided descriptions of parameters are very important. Most of the time `with_structured_output` is using a model's function/tool calling API, and you can effectively think of all of this information as being added to the model prompt.

In [10]:
# 我们让一个模型生成一个笑话，并将铺垫(setup)与笑点(punchline)分开。

from langchain_openai import ChatOpenAI
from typing import Optional
from pydantic import BaseModel,Field
from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env",verbose=True)

llm = ChatOpenAI(model="gpt-4o-mini")

# Pydantic
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke.")
    punchline: str = Field(description="The punchline of the joke.")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10."
    )

structured_llm = llm.with_structured_output(Joke)
structured_llm.invoke("Tell me a joke about apple.")
    

Joke(setup='Why did the apple stop in the middle of the road?', punchline='Because it ran out of juice!', rating=7)

当范式很复杂时，可以提供少样本示例（Few-shot prompting），其实就是给大模型举个例子：

In [8]:
from langchain_core.prompts import ChatPromptTemplate

system = """You are a hilarious comedian. Your specialty is knock-knock jokes. \
Return a joke which has the setup (the response to "Who's there?") and the final punchline (the response to "<setup> who?").

Here are some examples of jokes:

example_user: Tell me a joke about planes
example_assistant: {{"setup": "Why don't planes ever get tired?", "punchline": "Because they have rest wings!", "rating": 2}}

example_user: Tell me another joke about planes
example_assistant: {{"setup": "Cargo", "punchline": "Cargo 'vroom vroom', but planes go 'zoom zoom'!", "rating": 10}}

example_user: Now about caterpillars
example_assistant: {{"setup": "Caterpillar", "punchline": "Caterpillar really slow, but watch me turn into a butterfly and steal the show!", "rating": 5}}"""

prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{input}")])

few_shot_structured_llm = prompt | structured_llm
few_shot_structured_llm.invoke("what's something funny about woodpeckers")

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a hilarious comedian. Your specialty is knock-knock jokes. Return a joke which has the setup (the response to "Who\'s there?") and the final punchline (the response to "<setup> who?").\n\nHere are some examples of jokes:\n\nexample_user: Tell me a joke about planes\nexample_assistant: {{"setup": "Why don\'t planes ever get tired?", "punchline": "Because they have rest wings!", "rating": 2}}\n\nexample_user: Tell me another joke about planes\nexample_assistant: {{"setup": "Cargo", "punchline": "Cargo \'vroom vroom\', but planes go \'zoom zoom\'!", "rating": 10}}\n\nexample_user: Now about caterpillars\nexample_assistant: {{"setup": "Caterpillar", "punchline": "Caterpillar really slow, but watch me turn into a butterfly and steal the show!", "rating": 5}}'), additional_kw

实现 structured_outputs 有多种形式：
* tool calling
* json_mode

可以通过指定 `method=` 参数来选择使用哪种方式：

`method: Literal["function_calling", "json_mode", "json_schema"] = "json_schema"`

In [11]:
# 这是一个使用 json_mode 的例子

from langchain_openai import ChatOpenAI
from typing import Optional
from pydantic import BaseModel,Field
from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env",verbose=True)

llm = ChatOpenAI(model="gpt-4o-mini")

# Pydantic
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke.")
    punchline: str = Field(description="The punchline of the joke.")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10."
    )

structured_llm = llm.with_structured_output(None, method="json_mode")

structured_llm.invoke(
    "Tell me a joke about cats, respond in JSON with `setup` and `punchline` keys"
)

{'setup': 'Why did the cat sit on the computer?',
 'punchline': 'Because it wanted to keep an eye on the mouse!'}

对于不支持 tool calling 和 JSON mode 的模型，只能是在提示词中规定范式，然后使用 out parser 从模型应答中提取格式化信息：

In [4]:
from typing import List

from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env",verbose=True)

llm = ChatOpenAI(model="gpt-4o-mini")

class Person(BaseModel):
    """Information about a person."""

    name: str = Field(..., description="The name of the person")
    height_in_meters: float = Field(
        ..., description="The height of the person expressed in meters."
    )


class People(BaseModel):
    """Identifying information about all people in a text."""

    people: List[Person]


# Set up a parser
parser = PydanticOutputParser(pydantic_object=People)
# 这个是 out parser 提供的一个很有用的功能 
# print(parser.get_format_instructions())

# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user query. Wrap the output in `json` tags\n{format_instructions}",
        ),
        ("human", "{query}"),
    ]
).partial(format_instructions=parser.get_format_instructions())

query = "Anna is 23 years old and she is 6 feet tall"

print(prompt.invoke({"query": query}).to_string())

chain = prompt | llm | parser
chain.invoke({"query": query})

System: Answer the user query. Wrap the output in `json` tags
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"Person": {"description": "Information about a person.", "properties": {"name": {"description": "The name of the person", "title": "Name", "type": "string"}, "height_in_meters": {"description": "The height of the person expressed in meters.", "title": "Height In Meters", "type": "number"}}, "required": ["name", "height_in_meters"], "title": "Person", "type": "object"}}, "description": "Identifying information about all people in a text.", "properties": {"people": {"items"

People(people=[Person(name='Anna', height_in_meters=1.8288)])

当然，也可以使用自定义的 parser：

In [ ]:
import json
import re
from typing import List

from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env",verbose=True)

llm = ChatOpenAI(model="gpt-4o-mini")

class Person(BaseModel):
    """Information about a person."""

    name: str = Field(..., description="The name of the person")
    height_in_meters: float = Field(
        ..., description="The height of the person expressed in meters."
    )


class People(BaseModel):
    """Identifying information about all people in a text."""

    people: List[Person]


# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user query. Output your answer as JSON that  "
            "matches the given schema: \`\`\`json\n{schema}\n\`\`\`. "
            "Make sure to wrap the answer in \`\`\`json and \`\`\` tags",
        ),
        ("human", "{query}"),
    ]
).partial(schema=People.model_json_schema())


# Custom parser
def extract_json(message: AIMessage) -> List[dict]:
    """Extracts JSON content from a string where JSON is embedded between \`\`\`json and \`\`\` tags.

    Parameters:
        text (str): The text containing the JSON content.

    Returns:
        list: A list of extracted JSON strings.
    """
    text = message.content
    # Define the regular expression pattern to match JSON blocks
    pattern = r"\`\`\`json(.*?)\`\`\`"

    # Find all non-overlapping matches of the pattern in the string
    matches = re.findall(pattern, text, re.DOTALL)

    # Return the list of matched JSON strings, stripping any leading or trailing whitespace
    try:
        return [json.loads(match.strip()) for match in matches]
    except Exception:
        raise ValueError(f"Failed to parse: {message}")

query = "Anna is 23 years old and she is 6 feet tall"
print(prompt.format_prompt(query=query).to_string())

chain = prompt | llm | extract_json
chain.invoke({"query": query})